In [13]:
!pip install -U openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/37/34/f3c3d6bdc3eebf1b6a7c696dd6f934630af6cf5250cec099edf117cd3b53/openai-1.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for distro<2,>=1.7.0 from https://files.pythonhosted.org/packages/12/b3/231ffd4ab1fc9d679809f356cebee130ac7daa00d6d6f3206dd4fd137e9e/distro-1.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3,>=1.9.0 from https://files.pythonhosted.org/packages/e4/37/3ffe6e7daa1ea1b4bf5228807a92ccbae538cf57c0c50b93564c310c11a8/pydantic-2.6.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/81.8 kB ? eta -:--:--
     ---------------------------------------- 81.8/81.8 kB 4.5 MB/s eta 0:00:00
  Obtaining dependency in

In [34]:
import pandas as pd
import numpy as np
import json
from openai import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [16]:
df = pd.read_csv('./남사친_contents.csv')
df.head(5)

,Unnamed: 0,id,contents,comments,comments_num,date
0,0,1737026,나는 1달 있으면 전역하는 군인이고 여자친구는 사회인임5-6년 정도 알고 지내던 앤...,"['여자가 ㅈㄴ이상하네본문만 보면 fMRI라도 한번 찍어보고 싶다', '넌 선물 뭐...",66,2023-12-27 00:13:00
1,1,1732614,만으로 22살지금까지 스스로 자존감이 너무 없어서 연애 생각 없이 살아왔는데 이젠 ...,"['아주 좋은 자세야', '히히 고마워', '만이라고 하니까 헷갈리네', '01년생...",21,2023-12-16 19:21:00
2,2,1696594,지금 여자친구 집이고 같이 있는데.. 잠깐 화장실 갔다 왔는데 얼핏 여자친구 카톡 ...,"['ㅠ 나같으면 그냥은 못 넘어갈 거 같긴 해ㅠ', '힘내', '키톡 대화 수준이 ...",106,2023-09-27 04:13:00
3,3,1690602,난 경험이 있고 남자친구도 있대(오랜 남사친 관계였어서 연애사를 다 아는데 경험이 ...,['심리적인게 확실히 크긴 한데 잘 모르겠네 뭐가 원인인지.. 본인이 알텐데 허허'...,41,2023-09-13 17:10:00
4,4,1666599,"이성, 연애 관련 얘기인데내가 최근에 깨달은게 있는거같아상대방이 나에게 ""넌 조용한...","['글도 공대같아', '오 나도 이 말하려고 했는데ㅋㅋㅋ', '이정도면 공대가길 잘...",29,2023-07-20 16:23:00


In [22]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"comments":"'+row['comments']+'" }'
        fine_tuning_data.append({
            "messages": [
                {"role": "system", "content": "너는 친근한 서강대학교 재학생이야."},
                {"role": "user", "content": row['contents'] + row['comments']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [24]:
converted_data = convert_to_gpt35_format(df)

In [27]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

write_to_jsonl(converted_data, "converted_data.jsonl")



In [32]:

client = OpenAI(api_key=" ")

# Upload File
file_upload = client.files.create(
    file=open("converted_data.jsonl", "rb"), purpose="fine-tune"
)

# Create Fine-Tuning Job
suffix_name = "tutorial"
response = client.fine_tuning.jobs.create(
    training_file=file_upload.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)


In [33]:
response

FineTuningJob(id='ftjob-wf87ySVxXr2se0KfKDa8GuLm', created_at=1707133344, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-0rURo2pF7HEm1qVXGz3iwMyG', result_files=[], status='validating_files', trained_tokens=None, training_file='file-ClFTyb1CIVkW3bWqTrZ4b0jb', validation_file=None)

In [40]:
response = client.fine_tuning.jobs.retrieve("ftjob-wf87ySVxXr2se0KfKDa8GuLm")
response

FineTuningJob(id='ftjob-wf87ySVxXr2se0KfKDa8GuLm', created_at=1707133344, error=None, fine_tuned_model='ft:gpt-3.5-turbo-0613:personal:tutorial:8orplF1q', finished_at=1707134027, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-0rURo2pF7HEm1qVXGz3iwMyG', result_files=['file-VQiXXZL6DuBO2GV582kVan6Y'], status='succeeded', trained_tokens=428013, training_file='file-ClFTyb1CIVkW3bWqTrZ4b0jb', validation_file=None)

In [41]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0613:personal:tutorial:8orplF1q


In [35]:
def format_test(row):
    formatted_message = [{"role": "user", "content": row['contents']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content

def store_predictions(test_df, fine_tuned_model_id):
    test_df['Prediction'] = None
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")


In [64]:
test_df = pd.read_csv("./남사친_contents.csv")
test_df.shape
test_df

,Unnamed: 0,id,contents,comments,comments_num,date
0,0,1737026,나는 1달 있으면 전역하는 군인이고 여자친구는 사회인임5-6년 정도 알고 지내던 앤...,"['여자가 ㅈㄴ이상하네본문만 보면 fMRI라도 한번 찍어보고 싶다', '넌 선물 뭐...",66,2023-12-27 00:13:00
1,1,1732614,만으로 22살지금까지 스스로 자존감이 너무 없어서 연애 생각 없이 살아왔는데 이젠 ...,"['아주 좋은 자세야', '히히 고마워', '만이라고 하니까 헷갈리네', '01년생...",21,2023-12-16 19:21:00
2,2,1696594,지금 여자친구 집이고 같이 있는데.. 잠깐 화장실 갔다 왔는데 얼핏 여자친구 카톡 ...,"['ㅠ 나같으면 그냥은 못 넘어갈 거 같긴 해ㅠ', '힘내', '키톡 대화 수준이 ...",106,2023-09-27 04:13:00
3,3,1690602,난 경험이 있고 남자친구도 있대(오랜 남사친 관계였어서 연애사를 다 아는데 경험이 ...,['심리적인게 확실히 크긴 한데 잘 모르겠네 뭐가 원인인지.. 본인이 알텐데 허허'...,41,2023-09-13 17:10:00
4,4,1666599,"이성, 연애 관련 얘기인데내가 최근에 깨달은게 있는거같아상대방이 나에게 ""넌 조용한...","['글도 공대같아', '오 나도 이 말하려고 했는데ㅋㅋㅋ', '이정도면 공대가길 잘...",29,2023-07-20 16:23:00
5,5,1645620,밑에 글 보니까 나도 좀 궁금해서 글 써보려구우린 일단 20초 커플정리)여자친구가 ...,"['남친있는 여자한테 발렌타인 선물을 왜주지', '나도 어이없긴한데 그냥 원래 그런...",82,2023-06-02 11:30:00
6,6,1637787,"5년 넘게 잘 알고 지낸 사이에서 시작된 썸이었어요.서로 마음은 확인했는데,제가 수...","['하…….. 아니대체왜… 그만두시는게좋을것같아요… 가아니라 좋아요….', '아 제...",68,2023-05-17 00:16:00
7,7,1567128,http://www.ssodam.com/content/1566595?prev=2&p...,"['', '이거 탐나', '계속 좋은 사랑 하세요', '너 덕분에 할 수 있을거 같...",54,2022-12-28 01:46:00
8,8,1557588,남친이랑 만난지 3달 된 다미야남친이 주변 누나들이랑 친하더라고 학교 주식투자동아리...,"['ㅋㅋㅋ좀 이상한데', '묘하게 이상하지 않아..?', '이건 얘기해 봐야지', ...",66,2022-12-14 21:32:00
9,9,1498617,"문제까진 아니고남사친이 군대 휴가나왔대서 둘이 술먹으러 간다는데,,,,보내주는거 썩...","['둘이 술 마시는 건 싫어하는 티 낼 만한 일이지 않나..?', '친한 애라고 되...",37,2022-09-20 11:36:00


In [47]:
test_df = test_df[['contents']].iloc[:3]
test_df

,contents
0,나는 1달 있으면 전역하는 군인이고 여자친구는 사회인임5-6년 정도 알고 지내던 앤...
1,만으로 22살지금까지 스스로 자존감이 너무 없어서 연애 생각 없이 살아왔는데 이젠 ...
2,지금 여자친구 집이고 같이 있는데.. 잠깐 화장실 갔다 왔는데 얼핏 여자친구 카톡 ...


In [48]:
store_predictions(test_df, fine_tuned_model_id)

In [60]:
df_predict = pd.read_csv("./predictions.csv")
df_predict['Prediction'][2]

'{"comments":"[\'여자친구한테 물어봐\', \'그냥 물어봐\', \'물어봐\', \'물어봐'